# Robuste Programmierung mit Rust
## Idiomatische Programmiertechniken Anwenden
### Dorian A. Prill | FH Salzburg | Department IT
  
<img align="left" width="200" height="200" src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d5/Rust_programming_language_black_logo.svg/1024px-Rust_programming_language_black_logo.svg.png"/>
<img align="left" width="300" height="300" src="https://rustacean.net/assets/rustacean-orig-noshadow.svg"> 


# In diesem Kapitel behandeln wir

- Recap: Type Safety, Ownership, Borrowing

- Das Newtype-Pattern

- Das Typestate-Pattern

- Das Builder-Pattern

- Kombiniert als Typestate-Builder-Pattern

# Sonderfall Notebook

> **Achtung:** Aufgrund von Rust's strikten Regeln sind in diesem Notebook alle Zellen als Scoped-Block mit `{` und `}` eingeklammert, um die Definitionen und Variablen im Interpreter nach jeder Zelle zu zerstoeren um keine False-Positive Fehlermeldungen zu erzeugen.
Wenn Sie Code von hier kopieren, ersetzen Sie jeweils nur die aeusseren Scoped-Blocks durch eine Main-Funktion um den "freien" code (nicht die Defines `struct`, `enum`, etc).

# Recap: Ownership, Borrowing (1)



Wird dieses Programm kompilieren? Warum?

In [ ]:
{
    let s1 = String::from("Hello");
    let _s2 = s1; // underscore to suppress irrelevant (here) error
    println!("{}", s1);      
}


Das funktioniert nicht!  

> Erinnerung: In Rust kann jeder Wert nur einen Besitzer haben.   

Durch die Zuweisung `s2 = s1` wird der Besitz von `s1` an `s2` übergeben (ein sog. `move`, Ausnahme: *Primitive Types*).  
Die Übergabe an die Funktion `println!()` erfordert ein *Immutable Borrow*, dies kann aber nicht angefordert werden, da s1 der Wert nicht mehr gehört.

Stattdessen könnte man folgendes tun:

In [ ]:
{
    let s1 = String::from("hello");
    let s2 = s1.clone(); // kopie der daten mit neuer referenz, koennen nun unabh. genutzt werden.
    println!("s1 = {}, s2 = {}", s1, s2);
}

# Recap: Ownership, Borrowing (2)

Wird dieses Programm kompilieren? Warum?

In [ ]:
{
    let mut x = 5;
    let y = &mut x;     
    println!("{}", x);
    println!("{}", y)
}

Leider nicht! Es wird versucht, `x` sowohl als mutable als auch als immutable Referenz zu verwenden (es darf maximal eine immutable Referenz geben).
In diesem konstruierten Fall würde es also reichen, nur über die mutable Referenz zuzugreifen, da die Werte von `x`und `y` gleich sind:

In [ ]:
{
    let mut x = 5;
    let y = &mut x;
    // x = 6; zuweisung waehrend borrow ebenfalls nicht moeglich
    // nur über die mutable Referenz zugreifen
    println!("{}", y);    
}

# Recap: Type Safety (2)

In [ ]:
{
struct Cent(i32); // Newtype for representing amount in cent

enum Payment {
    Cash { amount: Cent },
    CreditCard { amount: Cent, number: String, expiry: String },
}
  
fn process_payment(payment: Payment) {
    match payment {
        Payment::Cash { amount: Cent(amount) } => println!("Processing cash payment of ${:.2}", amount as f64 / 100.0),
        Payment::CreditCard { amount: Cent(amount), number, expiry } => println!("Processing credit card payment of ${:.2} with card number: {} expiring: {}", amount as f64 / 100.0, number, expiry),
    }
}

    let my_payment = Payment::Cash { amount: Cent(5000) }; // 50.00 whole units
    let your_payment = Payment::CreditCard {
        amount: Cent(10000), // 100.00 whole units
        number: String::from("1234-5678-9012-3456"),
        expiry: String::from("12/24"),
    };

    process_payment(my_payment);
    process_payment(your_payment);
}

Ja, dies funktioniert fehlerfrei!  
1. Das Amount-Struct hilft z.B. zwischen Euro-Ints und Cent-Ints zu unterscheiden.
2. Exhausitve Pattern-Matching verhindert unerwünschte Fälle.

# Recap: Lifetimes (1)
Lifetimes sind eine ergänzende Taktik zu Ownership/Borrowing, um sicherzustellen, dass Referenzen auf gültige Werte zeigen.  
Nehmen wir folgendes Beispiel (Lifetimes werden mit `'` markiert, die Namen sind frei wählbar, oft einfach 'a, 'b, 'c, ...):

In [ ]:
{
    fn longest<'a>(s1: &'a str, s2: &'a str) -> &'a str {
        if s1.len() > s2.len() {
            s1
        } else {
            s2
        }
    }

    let string1 = String::from("Hello");
    //let result;
    {
        let string2 = String::from("World!");
        let result = longest(string1.as_str(), string2.as_str());
        println!("The longest string is {}", result);
    }
    
}


Der Compiler erkennt dank der Lifetime-Analyse, dass der String `s2` *ausserhalb* der Funktion nicht lange genug lebt!  
Wir muessen also den Main-Code aendern:

In [ ]:
{
    fn longest<'a>(s1: &'a str, s2: &'a str) -> &'a str {
        if s1.len() > s2.len() {
            s1
        } else {
            s2
        }
    }

    
    let string1 = String::from("Hello");
    let result;
    let string2 = String::from("World!");
    result = longest(string1.as_str(), string2.as_str());
    println!("The longest string is {}", result);
}

# Das Newtype-Pattern

Das Newtype-Pattern ist eine einfache aber effektive Programmiertechnik zur Erhöhung der Typensicherheit eines Programms, die in einigen Programmiersprachen (z.B. Rust, Haskell) unterstützt wird.  
Mit dem Newtype-Pattern wird ein neuer Typ definiert, der sich nur in der Bezeichnung von einem bestehenden Typ unterscheidet.

Typensicherheit wird oft unterschätzt, aber sie ist ein wichtiger Bestandteil der Softwareentwicklung [z.B. NASA Climate Orbiter Failure](https://en.wikipedia.org/wiki/Mars_Climate_Orbiter).

> Achtung: `typedef` in C/C++ ist kein Newtype-Pattern, da es nur ein Alias für einen bestehenden Typ ist und somit die normalen Konvertierungsregeln angewendet werden. In Rust ist es ein echter neuer Datentyp der nach allen Regeln geprüft wird.

# Newtype-Pattern in Rust

In [ ]:
{
struct Meters(f64);
struct Feet(f64);

impl From<Meters> for Feet {
    fn from(meters: Meters) -> Feet {
        Feet(meters.0 * 3.28084)
    }
}
impl From<Feet> for Meters {
    fn from(feet: Feet) -> Meters {
        Meters(feet.0 / 3.28084)
    }
}
let distance_in_meters = Meters(100.0); // 100 meters
let distance_in_feet: Feet = distance_in_meters.into();
println!("Distance in feet: {:.2}", distance_in_feet.0);
let converted_back_to_meters: Meters = distance_in_feet.into();
println!("Converted back to meters: {:.2}", converted_back_to_meters.0);
}

> Achtung: In C kann ein `struct` mit einer Variable und `get`/`set`-Methoden ebenfalls nicht die gleichen Compile-Zeit Sicherheiten wie Rust bieten.

Dies hat mehrere Gründe:
- Fehlende Ownership/Borrowing Regeln
- Weniger striktes Typsystem
- Keine statische Analyse für Feldzugriff (Rust hat private by default, pub und pub(crate))

## Mehrere Möglichkeiten für Newtypes

Zuvor haben wir nur eine - wenn auch häufig angewandte - Möglichkeit für das Newtype-Pattern gesehen. Es gibt aber verschiedene Möglichkeiten: 

1. **Tuple Structs (Anonyme Structs)**: Simple Benutzung für `struct`s mit nur einem Feld. Siehe Beispiel zuvor.
2. **Named Structs**: "Normale Benutzung von `struct`s mit benannten Feldern. Erhöht die Klarheit für 2 und mehr Felder.
3. **Unit Structs**: Ein Größenloser Typ der zum markieren von Zuständen zur Compile-Zeit verwendet werden kann. Diese Technik werden wir folgend noch genauer kennenlernen.
4. **Phantom Data und Generics**: Variante 1 oder 2 und ein ein zusätzliches, größenloses `std::marker::PhantomData`Feld um Information über einen generischen Typ zu prüfen ohne diesen selbst im struct zu halten.

## Newtypes mit Named Structs

In [ ]:
{
struct Cent {
    value: i32, // Alle Felder sind benannt
}

impl Cent {
    pub fn new(value: i32) -> Cent {
        Cent { value }
    }
    
    pub fn value(&self) -> i32 {
        self.value  // named access
    }
}
}

## Newtypes mit Unit Structs

Ein Unit Struct hat keine Größe zur Laufzeit (Zero-Sized-Type), da es nur zum prüfen des Typsystems dient. Dies ist in manchen Fällen sinnvoll, wie etwa der Anwendung des Typestate-Pattern, weswegen wir dieses Beispiel jetzt erstmal nach hinten stellen.   

Zumindest können wir beweisen, dass der Typ keinen Speicherplatz einnimmt:

In [ ]:
{
struct MyUnitStruct;

println!("Size of MyUnitStruct: {}", std::mem::size_of::<MyUnitStruct>());
}

## Newtypes mit PhantomData und Generics

In komplexeren Anwendungen, welchen von generischen Datentypen abhängen, kann man ein sog. Marker-Feld ebenfalls als Teil des `struct`verwenden um den Datentypen zur Compile-Zeit zu verfolgen, ohne das Datum selbst jemals zu speichern.  
Damit kann z.B. das Lifetime-Tracking eines Raw Pointers auf nicht-eigene Daten ermöglicht werden. Dies wird angewendet, wenn man sichere Abstraktionen um unsichere Programmteile bauen möchte.

In [ ]:
{
use std::marker::PhantomData;

// T selbst wird nie gespeichert, aber die statische 
// Analyse (auch Lifetime) wird trotzdem durchgefuehrt!
struct Cent<T> {
    value: i32,
    _marker: PhantomData<T>,
} 

impl<T> Cent<T> {
    pub fn new(value: i32) -> Cent<T> {
        Cent { value, _marker: PhantomData }
    }
}
}

# Das Typestate-Pattern: Motivation

- In C ist es beispielsweise möglich eine Datei zu schließen, und danach wieder versuchen zu schreiben, ohne, dass der Compiler es merkt (und was zu einem Laufzeitfehler führt).
```c
FILE *file = fopen("data.txt", "r");
fclose(file);
// Fehler: Schreiben nach dem Schließen der Datei, aber keine Kompilierfehler
fwrite(data, sizeof(char), length, file);
```
- So ein offensichtlicher Fehler sollte doch schon zur Compile-Zeit irgendwie abgefangen werden können!
- Wir müssten also den Zustand in einem eigenen Datentyp abbilden, sodass er von der statischen Analyse erfasst wird
- Mit diesen Datentypen könnten wir dann sogar die API-Oberfläche begrenzen, dass sie gar nicht falsch benutzt werden *kann*

# Typestate-Pattern: Einführung

Während in C keine statische Information zum Öffnungszustand besteht, können wir in Rust diese Zustände abbilden.

```Rust
struct OpenFile { /* ... */ }
struct ClosedFile { /* ... */ }

impl OpenFile {
    fn close(self) -> ClosedFile { // Beachte: self statt &self
        // Übergang von geöffnetem zu geschlossenem Zustand
        ClosedFile { /* ... */ }
    }
}

impl ClosedFile {
    fn open(self) -> OpenFile { // Beachte: self statt &self
        // Übergang von geschlossenem zu geöffnetem Zustand
        OpenFile { /* ... */ }
    }
}

let file = OpenFile { /* ... */ };
let closed_file = file.close();
// file kann ab hier nicht mehr benutzt werden, Rust verbietet sonst die Kompilierung!
```

Dadurch entfallen auch kostspielige Laufzeit-Checks (`if`/`switch`), unser Program ist dadurch also sogar effizienter.
> Das Ziel ist, *unzulässige Zustände unrepresentierbar zu machen*.
In C können wir dies nicht abbilden, auch nicht mit dem Struct Ansatz, da:

1. C kennt keine Ownership/Borrowing Regeln. Da Rust bei einem Zustandsübergang ein Nutzen des alten Zustands verhindert.
2. In C ist sind Zeiger auf den alten Zustand weiterhin Zugreifbar, es gibt keine Compile-Zeit Garantien für Speichernutzung.

[Typestate Programming im Rust Embedded Book](https://docs.rust-embedded.org/book/static-guarantees/typestate-programming.html)

## Generic Typestate-Pattern

Im vorherigen Beispiel mussten Felder noch zwischen den zwei Datentypen dupliziert werden, was nicht ideal ist.  

Stattdessen kann man ein gemeinsames `struct` verwenden, was über einen generischen Parameter verfügt, den wir für den Zustand benutzen.

Als erstes führen wir ein *leeres* Trait ein, welches Später den generischen Parameter füllen wird.  
Dann erstellen wir *ebenfalls leere* `structs`, welche die einzelnen Zustände repräsentieren.  
Zuletzt müssen wir noch das Trait implementieren (*meist ebenfalls leer*).

```Rust
// Define a trait to represent any state
pub trait State {}
// Define the different states
pub struct OpenState;
pub struct ClosedState;
// Implement the State trait for the states
impl State for OpenState {}
impl State for ClosedState {}

// Generic File struct, where the state is represented by the generic parameter S
pub struct File<S: State> {
    // Fields like file descriptor etc
}

Jetzt müssen wir natürlich - wie zuvor - noch die einzelnen Zustände und ihre Übergänge implementieren:

```Rust
impl File<OpenState> {
    // Method to close the file, transitioning from OpenState to ClosedState
    pub fn close(self) -> File<ClosedState> {
        // Transition logic from open to closed
        File {
            // File fields go here
        }
    }
    pub fn read(&self) { } // Non consuming ref
}

impl File<ClosedState> {
    pub fn open(self) -> File<OpenState> {
        // Transition logic from closed to open
        File {
            // File fields go here
        }
    }
}
```

So sieht dann der Aufrufende Code aus. Auch hier schützt das Typsystem wieder vor fehlerhaften Zuständen:

```Rust
fn main() {
    // Create an open file
    let open_file: File<OpenState> = File {
        // Initialize file fields
    };
    // Use the file while it's open
    open_file.read();
    // Close the file
    let closed_file: File<ClosedState> = open_file.close();

    closed_file.read() // error, function doesn't exist for File<ClosedState>

    // Reopen the file
    let reopened_file: File<OpenState> = closed_file.open();
}
```

# Das Builder-Pattern: Motivation

- Das Builder-Pattern ist ein Entwurfsmuster, das verwendet wird, um die Erstellung von Objekten mit optionalen Feldern zu vereinfachen.
- Zum Beispiel bei der Erstellung von GUI-Elementen, oder bei der Erstellung von SQL-Queries.
- In Rust ist das Builder-Pattern besonders nützlich, da es die Verwendung von `Option`-Typen und `unwrap()`-Aufrufen vermeidet.
- Es wird oft in Kombination mit dem Typestate-Pattern verwendet, um die Erstellung von Objekten in einem bestimmten Zustand zu erzwingen (Typestate-Builder).

## Builder-Pattern: Funktionsweise

1. Sie wollen ein Object `MyObject` erstellen, das mehrere Felder hat, von denen einige optional sind.
2. `MyObject` erhält nun einen Partnertyp `MyObjectBuilder`, welcher die *gleichen Felder* wie `MyObject` enthält.
3. Wir dürfen `MyObject` nur über `MyObjectBuilder` erstellen, da `MyObject` keine öffentlichen Konstruktoren hat.
4. `MyObjectBuilder` hat für *jedes* optionale Feld eine Methode, die das Feld setzt und das neue `MyObjectBuilder`-Objekt zurückgibt.
5. Eine finale methode `build()` erstellt das `MyObject`-Objekt aus dem `MyObjectBuilder`-Objekt.

In [ ]:
{
struct Circle { // private
    x: f64,
    y: f64,
    radius: f64,    
}
pub struct CircleBuilder { // public
    x: f64,
    y: f64,
    radius: f64,
}

impl CircleBuilder {
    pub fn new() -> CircleBuilder { // can take mandatory parameters
        CircleBuilder { x: 0.0, y: 0.0, radius: 1.0 }
    }
    pub fn x(&mut self, coordinate: f64) -> &mut CircleBuilder {
        self.x = coordinate;
        self
    }
    pub fn y(&mut self, coordinate: f64) -> &mut CircleBuilder {
        self.y = coordinate;
        self
    }
    pub fn radius(&mut self, radius: f64) -> &mut CircleBuilder {
        self.radius = radius;
        self
    }
    pub fn build(&self) -> Circle {
        Circle { x: self.x, y: self.y, radius: self.radius }
    }
}
let c = CircleBuilder::new()
    .x(1.0)
    .y(2.0)
    .radius(2.5)
    .build();
println!("Circle at ({}, {}), radius: {}", c.x, c.y, c.radius);
}

## Builder-Pattern: Warum nicht einfach ein Konstruktor mit `Option`s?

Eine einfachere Funktion mit `Option`-Typen, z.B. so: 
```Rust
fn new(field1: Option<i32>, field2: Option<String>, field3: Option<f64>) -> MyObject {
    MyObject { 
        field1: field1.unwrap_or(1), 
        field2: field2.unwrap_or(0), 
        field3: field3.unwrap_or(1) 
    }
}
```

würde auch funktionieren, aber:

- Die Lesbarkeit nimmt ab: Beim Aufruf von Funktionen mit vielen Parametern ist nicht schnell ersichtlich, welche Parameter es sind.
- Es ist nicht möglich, die Reihenfolge der Parameter zu ändern.
- Es ist nicht möglich, die Parameter zu benennen.
- Es ist nicht möglich, die Parameter einzeln zu validieren.
- Es ist nicht möglich, die Parameter zu verändern, ohne die Signatur der Funktion zu verändern (*breaking change*).
  

# In Kombination als Typestate-Builder Pattern

Wir können jetzt also:

1. Zustände zur Compile Zeit beschreiben und prüfen
2. Objekte erzeugen und partiell belegen

Wenn wir dies nun kombinieren, können wir Zustände an unterschiedliche Stufen der Initialisierung binden!

- Verbesserte API Ergonomie und Sicherheit
- Mehr Typen und Code -> höhere Komplexität
- Wartungsaufwand: Änderungen in mehreren Typen und Zustandsübergängen

# Beispiel: File Transfer API

Starten wir wieder mit der Beschreibung der Marker-Typen für die Zustände:

In [ ]:
struct Connected;
struct Authenticated;
struct Transferring;

Wir wollen eine `Session`beschreiben, die von diesen Zuständen abhängt (aber ihn nicht enthält):

In [ ]:
struct Session<State> {
    connection_info: String,
    state: std::marker::PhantomData<State>,
}

Jetzt müssen wir die Zustandsübergänge definieren:

1. Eine Verbindung herstellen und authentifizieren (könnten auch zwei Zustände sein):

In [ ]:
impl Session<Connected> {
    fn new(connection_info: &str) -> Self {
        Session {
            connection_info: connection_info.to_string(),
            state: std::marker::PhantomData,
        }
    }

    fn authenticate(self, credentials: &str) -> Session<Authenticated> {
        // Perform authentication logic here
        println!("Authenticating with credentials: {}", credentials);
        Session {
            connection_info: self.connection_info,
            state: std::marker::PhantomData,
        }
    }
}


2. Die Dateiübertragung anfragen:

In [ ]:
impl Session<Authenticated> {
    fn start_transfer(self) -> Session<Transferring> {
        // Initialize file transfer logic here
        println!("Starting file transfer...");
        Session {
            connection_info: self.connection_info,
            state: std::marker::PhantomData,
        }
    }
}


3. Datei senden und Transfer beenden

In [ ]:
impl Session<Transferring> {
    fn send_file(&self, file_path: &str) {
        // File sending logic here
        println!("Sending file: {}", file_path);
    }

    fn end_transfer(self) {
        // Cleanup logic here
        println!("Ending file transfer.");
    }
}


So benutzen wir die API dann in der Praxis:

In [ ]:

let session = Session::<Connected>::new("server_address")
    .authenticate("user_credentials")
    .start_transfer();

session.send_file("/path/to/file");
session.end_transfer();


Wir werden vor Fehlzugriffen geschützt:

In [ ]:
let session = Session::<Connected>::new("server_address");
session.send_file("/path/to/file"); // Compile-time error!


# Übung: State-Machine in einem Kommunikationsprotokoll

Wir wollen ein einfaches Netzwerk-Protokoll schreiben - Zum Glück gibt es schon einfache Protokolle wie UDP. Leider bietet UDP aber keinen Acknowledge/Resend Mechanismus wie TCP.  
Wir wollen jetzt diesen Mechanismus auf UDP aufsetzen, dazu müssen wir einen Zustandsautomat für das Protokoll definieren.

**Sender States:**

- Idle: Waiting for Message
- Receiving: Receiving Message
- AwaitingAck: Wait for Acknowledgement of Message
- Resend: Resend Message after Timeout or NACK


**Receiver States:**

- Idle: Waiting to send Message
- Sending: Sending Message
- SendingAck: Acknowledge the Reception of a Message
- SendingNAck: Do not acknowledge the Reception of a Message (i.e. malformed message)